In [ ]:
%pip install plyer -q
%pip install requests -q 
%pip install schedule -q
%pip install pync -q

In [6]:
import requests 
import time
import schedule 
import threading
from pync import Notifier
import json
import sys
from typing import Tuple, Optional
import logging
from plyer import notification

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
DEFAULT_CONFIG = {
    "api_key": "Replace with your actual API key",  
    "city": "Greater Noida",
    "notification_interval_minutes": 1, #every minute for quick testing!
    "platform": "mac", #or windows, linux
}

In [10]:
def load_config(config_file="config.json"):
    try:
        with open(config_file, "r") as f:
            config = json.load(f)
        #check if all required keys are present, and if not, add them with the default value
        for key, value in DEFAULT_CONFIG.items():
            if key not in config:
                config[key] = value
        return config
    except FileNotFoundError:
        logging.warning("Config file not found. Creating default config.")
        with open(config_file, "w") as f:
            json.dump(DEFAULT_CONFIG, f, indent=4)
        return DEFAULT_CONFIG
    except json.JSONDecodeError:
        logging.error("Invalid JSON in config file.")
        return DEFAULT_CONFIG

In [12]:
def save_config(config, config_file="config.json"):
    with open(config_file, "w") as f:
        json.dump(config, f, indent=4)

In [14]:
def get_weather(api_key: str, city: str) -> Optional[Tuple[float, int, int, str]]:
    base_url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    try:
        response = requests.get(base_url)
        response.raise_for_status()
        weather_data = response.json()

        main = weather_data['main']
        temp = main['temp']
        pressure = main['pressure']
        humidity = main['humidity']
        weather_report = weather_data['weather']
        weather_description = weather_report[0]['description']

        return temp, pressure, humidity, weather_description
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            logging.error(f"City not found: {city}")
        elif e.response.status_code == 401:
            logging.error("Invalid API key.")
        else:
            logging.error(f"HTTP error: {e}")
        return None
    except requests.exceptions.RequestException as e:
        logging.error(f"Error fetching weather data: {e}")
        return None

In [16]:
def show_notification(config: dict):
    weather_info = get_weather(config["api_key"], config["city"])
    if weather_info:
        temp, pressure, humidity, description = weather_info

        notification_message = (
            f"Weather in {config['city']}:\n"
            f"Temperature: {temp}°C\n"
            f"Pressure: {pressure} hPa\n"
            f"Humidity: {humidity}%\n"
            f"Description: {description}"
        )

        if config["platform"].lower() == "mac":
            Notifier.notify(notification_message, title="Weather Update")
        else:
            plyer.notification.notify(
                title="Weather Update",
                message=notification_message,
                timeout=10,
            )
    else:
        if config["platform"].lower() == "mac":
            Notifier.notify("Failed to fetch weather data.", title="Error")
        else:
            plyer.notification.notify(
                title="Error",
                message="Failed to fetch weather data.",
                timeout=10,
            )

In [18]:
def run_schedule(config: dict):
    global running
    schedule.every(config["notification_interval_hours"]).minutes.do(lambda: show_notification(config))
    while running and schedule.jobs:
        schedule.run_pending()
        time.sleep(1)
running = True
config = load_config()
scheduler_thread = threading.Thread(target=lambda: run_schedule(config), daemon=True)
scheduler_thread.start()

In [ ]:
def stop_notifications():
    global running
    running = False
    schedule.clear()
    logging.info("Weather notifications stopped.")

if __name__ == "__main__":
    city = None
    interval = None
    args = sys.argv[1:]
    
    i = 0
    while i < len(args):
        if args[i] == "--city":
            if i + 1 < len(args):
                city = args[i + 1]
                i += 2
            else:
                logging.warning("Missing city value.")
                i += 1
        elif args[i] == "--interval":
            if i + 1 < len(args):
                try:
                    interval = int(args[i + 1])
                    i += 2
                except ValueError:
                    logging.warning("Invalid interval value.")
                    i += 1
            else:
                logging.warning("Missing interval value.")
                i += 1

        else:
            i +=1

    if city:
        config["city"] = city
        save_config(config)
    if interval:
        config["notification_interval_minutes"] = interval #changed to minutes
        save_config(config)

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        stop_notifications()
        logging.info("Weather notifier stopped.")